# Inferring
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup

In [1]:
import yaml

with open("config.yml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [9]:
import openai
import cohere
openai.api_key  = config['openai_key']
co = cohere.Client(config['cohere_key'])

In [10]:
def get_completion(prompt, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def cohere_complete(prompt, model= "command-nightly"):
    response = co.generate(
        model=model,
        prompt=prompt,
        max_tokens=2500,
        temperature=0
    )
    return response.generations[0].text

## Product review text

In [11]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [12]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response_openai = get_completion(prompt)
print(response_openai)

print("\n")

response_cohere = cohere_complete(prompt)
print(response_cohere)

The sentiment of the product review is positive.


The sentiment of the product review is positive.


In [13]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response_openai = get_completion(prompt)
print(response_openai)

print("\n")

response_cohere = cohere_complete(prompt)
print(response_cohere)

positive


positive


## Identify types of emotions

In [16]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response_openai = get_completion(prompt)
print(response_openai)

print("\n")

response_cohere = cohere_complete(prompt)
print(response_cohere)

happy, satisfied, grateful, impressed, content


needed, had, got, easy, great


## Identify anger

In [17]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response_openai = get_completion(prompt)
print(response_openai)

print("\n")

response_cohere = cohere_complete(prompt)
print(response_cohere)

No


no


## Extract product and company name from customer reviews

In [18]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response_openai = get_completion(prompt)
print(response_openai)

print("\n")

response_cohere = cohere_complete(prompt)
print(response_cohere)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


Item: Lamp
Brand: Lumina


## Doing multiple tasks at once

In [19]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response_openai = get_completion(prompt)
print(response_openai)

print("\n")

response_cohere = cohere_complete(prompt)
print(response_cohere)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp with additional storage",
  "Brand": "Lumina"
}


{
    "Sentiment": "positive",
    "Anger": false,
    "Item": "lamp",
    "Brand": "Lumina"
}


## Inferring topics

In [20]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

## Infer 5 topics

In [21]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response_openai = get_completion(prompt)
print(response_openai)

print("\n")

response_cohere = cohere_complete(prompt)
print(response_cohere)

government survey, job satisfaction, NASA, Social Security Administration, employee concerns


survey, NASA, Social Security Administration, job satisfaction


In [23]:
print(response_openai.split(sep=','))

print("\n")

print(response_cohere.split(sep=','))


['government survey', ' job satisfaction', ' NASA', ' Social Security Administration', ' employee concerns']


['survey', ' NASA', ' Social Security Administration', ' job satisfaction']


In [24]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

## Make a news alert for certain topics

In [27]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response_openai = get_completion(prompt)
print(response_openai)

print("\n")

response_cohere = cohere_complete(prompt)
print(response_cohere)

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


nasa, local government, engineering, employee satisfaction, federal government


In [28]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response_openai.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

print("\n")

topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response_cohere.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!




IndexError: list index out of range

## Try experimenting on your own!